# Importing Libraries

In [1]:
import numpy as np
from tensorflow.keras import regularizers
import math
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.datasets import cifar100
from keras import backend as K
import cv2
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# Loading Data

In [2]:
# Download dataset from tensorflow repository
(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='coarse')

169001437/169001437 [==============================] - 2s 0us/step


In [3]:
print(f"X Train Shape: {X_train}")
print(f"X Test Shape: {X_test}")
print(f"y Train Shape: {y_train}")
print(f"y Test Shape: {y_test}")

(50000, 32, 32, 3) (50000, 1)


# Change Images Size

In [4]:
X_train = np.array([cv2.resize(img, (140, 140)) for img in X_train])
X_test = np.array([cv2.resize(img, (140, 140)) for img in X_test])

# Encoding Target Value

In [6]:
enc = OneHotEncoder()
y_train=enc.fit_transform(y_train).toarray().astype(int)
y_test=enc.transform(y_test).toarray().astype(int)

print(y_train.shape)
print(y_train[0])

(50000, 20)
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


# Data Augmentation

In [7]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(tfl.RandomFlip('horizontal'))
    data_augmentation.add(tfl.RandomRotation(0.2))

    return data_augmentation
data_augmentation = data_augmenter()

# AlexNet architecture 

In [9]:
def arch(input_shape, data_augmentation):

    input_img = tf.keras.Input(shape=input_shape)
    layer = data_augmenter()(input_img)
    layer=tfl.Conv2D(filters= 96 , kernel_size= 11,strides=(4, 4))(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)
    
    layer=tfl.Conv2D(filters= 256 , kernel_size= 5 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Conv2D(filters= 384 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.Conv2D(filters= 384 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.Conv2D(filters= 256 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Flatten()(layer)
    
    layer=tfl.Dense(units=4096, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Dense(units=4096, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Dense(units=1000, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    outputs=tfl.Dense(units= 20 , activation='softmax')(layer)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    return model

# training and evaluating the model

In [12]:
conv_model = arch((140, 140, 3),data_augmentation)
conv_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
conv_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 140, 140, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 140, 140, 3)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 33, 33, 96)        34944     
                                                                 
 re_lu_5 (ReLU)              (None, 33, 33, 96)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 17, 17, 96)       0         
 2D)                                                             
                                                                 
 batch_normalization_6 (Batc  (None, 17, 17, 96)       384       
 hNormalization)                                           

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)
history = conv_model.fit(train_dataset,epochs=25,validation_data=test_dataset,batch_size=64,shuffle=True)

Epoch 1/25
782/782 [==============================] - 48s 55ms/step - loss: 2.7394 - accuracy: 0.1922 - val_loss: 2.4167 - val_accuracy: 0.2735
Epoch 2/25
782/782 [==============================] - 41s 53ms/step - loss: 2.3616 - accuracy: 0.2799 - val_loss: 2.1809 - val_accuracy: 0.3424
Epoch 3/25
782/782 [==============================] - 42s 53ms/step - loss: 2.1579 - accuracy: 0.3361 - val_loss: 2.0034 - val_accuracy: 0.3885
Epoch 4/25
782/782 [==============================] - 41s 53ms/step - loss: 2.0068 - accuracy: 0.3767 - val_loss: 1.9193 - val_accuracy: 0.4136
Epoch 5/25
782/782 [==============================] - 42s 53ms/step - loss: 1.8794 - accuracy: 0.4147 - val_loss: 1.8012 - val_accuracy: 0.4515
Epoch 6/25
782/782 [==============================] - 41s 53ms/step - loss: 1.7670 - accuracy: 0.4499 - val_loss: 1.7086 - val_accuracy: 0.4750
Epoch 7/25
782/782 [==============================] - 41s 53ms/step - loss: 1.6658 - accuracy: 0.4801 - val_loss: 1.6371 - val_accuracy: